In [2]:
import torch
from transformers import AutoModel
from torch.utils.data import DataLoader
from torchvision import transforms
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import transformers
from datasets import load_dataset

import random
import random
from PIL import ImageDraw, ImageFont, Image
import pathlib
import sklearn
import datasets
import pandas as pd
import sklearn.preprocessing
import sklearn.model_selection
import glob
import functools

TOKENIZERS_PARALLELISM=False

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [27]:
device = 'cpu'

dataset_path = '/Users/tylerklimas/Desktop/BERTModel/dataset_processed'
dataset_raw = datasets.load_from_disk(dataset_path)

labels = dataset_raw['train'].features['label'].names
labels

id2label = {}
label2id = {}
for idx, ele in enumerate(labels):
    label2id[ele] = idx
    id2label[idx] = ele

    

In [28]:
base_model = "distilbert-base-uncased"
model_dir = '/Users/tylerklimas/Desktop/BERTModel/BERTModelArchive'


model = transformers.AutoModelForSequenceClassification.from_pretrained(model_dir, return_dict=False)
tokenizer = transformers.AutoTokenizer.from_pretrained(model_dir)

In [29]:
columns = set(dataset_raw['train'].column_names) - set(['text', 'label'])
columns

{'brand', 'item_id', 'item_name', 'main_image_id', 'node'}

In [30]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset_raw.map(tokenize_function, batched=True, remove_columns=columns)

In [16]:
subset = tokenized_datasets['train'].num_rows
subset = 20 # JUST FOR INFERENCE PURPOSE

test_dataset = tokenized_datasets['test'].shuffle(seed=42).select(range(subset))
test_dataset.set_format(type='torch')
test_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 20
})

In [31]:
import utils
utils.prediction_batch(model, test_dataset)   

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/datasets/load.py:753: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
100%|█████████████████████████████████████████████| 1/1 [00:05<00:00,  5.10s/it]


{'accuracy': 0.8}

In [32]:
quantized_model_int8 = torch.quantization.quantize_dynamic(
                       model,
                       {torch.nn.Linear},
                       dtype = torch.qint8)
# quantized_model_int8 

In [33]:
import os

def print_size_model(model):
    torch.save(model.state_dict(), 'temp.p')
    print('Size (MB): ', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')
    
print_size_model(model)
print_size_model(quantized_model_int8)

Size (MB):  267.942302
Size (MB):  138.729314


In [34]:
base_model = "distilbert-base-uncased"
script_tokenizer = transformers.AutoTokenizer.from_pretrained(model_dir, torchscript=True)
script_model = transformers.AutoModelForSequenceClassification.from_pretrained(model_dir,
                                                                        num_labels = len(labels),
                                                                        label2id=label2id,
                                                                        id2label=id2label,
                                                                        torchscript=True)                                               
                                                                               
                                                                               

In [35]:
text = 'mens dress shoes'

res = script_tokenizer.encode_plus(text, return_tensors='pt', padding='max_length', truncation=True)

text_tokens = res['input_ids'].to(device)
text_attentions = res['attention_mask'].to(device)

dummy_input = [text_tokens, text_attentions]

script_model = script_model.to(device)

traced_model = torch.jit.trace(script_model, dummy_input)



/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/transformers/modeling_utils.py:4193: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/transformers/models/distilbert/modeling_distilbert.py:246: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  mask, torch.tensor(torch.finfo(scores.dtype).min)


In [36]:
model_dir = '/Users/tylerklimas/Desktop/BERTModel/BERTModelArchive'
model.save_pretrained('/Users/tylerklimas/Desktop/BERTModel/BERTModelArchive')
tokenizer.save_pretrained('/Users/tylerklimas/Desktop/BERTModel/TBERTModelArchive')
torch.save(model.state_dict(), '/Users/tylerklimas/Desktop/BERTModel/TrainedModels/model_weights.pt')



In [37]:
setup_config = {
    'model_name':"pt-original",
    "do_lower_case": tokenizer.do_lower_case,
    "num_labels":len(id2label),
    'save_mode':"original",
    'max_length': tokenizer.model_max_length,
    'captum_explanation': True,
    "base_model": 'distilbert-base-uncased',
    'top_k': 5
}

In [38]:
import json

with open(f'{model_dir}/setup_config.json', 'w') as f:
    json.dump(setup_config,f)

In [39]:
setup_config_trace = {**setup_config}
setup_config_trace['model_name'] = 'pt-jit'
setup_config_trace['capture_explanation'] = False
setup_config_trace['save_model'] = 'jit'


In [40]:

with open(f'{model_dir}/setup_config.json', 'w') as f:
    json.dump(setup_config_trace,f)